<a href="https://colab.research.google.com/github/fecsaba/ComfyUI/blob/main/Flux1_dev_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Notes
- Uncheck `UPDATE_COMFY_UI`, and `USE_COMFYUI_MANAGER` and `INSTALL_GGUF_NODE` after first run if `USE_GOOGLE_DRIVE` is checked.


### Install Dependencies

In [ ]:
# #@title Environment Setup

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_GGUF_NODE = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_GGUF_NODE'] = INSTALL_GGUF_NODE
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f ".ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/nightly/windows_base_files/run_nvidia_gpu.bat" ] && chmod 755 .ci/nightly/windows_base_files/run_nvidia_gpu.bat
  ![ -f ".ci/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows/update.py" ] && chmod 755 .ci/update_windows/update.py
  ![ -f ".ci/update_windows/update_comfyui.bat" ] && chmod 755 .ci/update_windows/update_comfyui.bat
  ![ -f ".ci/update_windows/README_VERY_IMPORTANT.txt" ] && chmod 755 .ci/update_windows/README_VERY_IMPORTANT.txt
  ![ -f ".ci/update_windows/run_cpu.bat" ] && chmod 755 .ci/update_windows/run_cpu.bat
  ![ -f ".ci/update_windows/run_nvidia_gpu.bat" ] && chmod 755 .ci/update_windows/run_nvidia_gpu.bat

  !git pull

!echo -= Install dependencies =-
!pip3 install accelerate
!pip3 install einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip3 install torchsde
!pip3 install kornia>=0.7.1 spandrel soundfile sentencepiece

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f "ComfyUI-Manager/check.sh" ] && chmod 755 ComfyUI-Manager/check.sh
  ![ -f "ComfyUI-Manager/scan.sh" ] && chmod 755 ComfyUI-Manager/scan.sh
  ![ -f "ComfyUI-Manager/node_db/dev/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-win.bat" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat

  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull


if OPTIONS['INSTALL_GGUF_NODE']:
  %cd $WORKSPACE
  %cd custom_nodes
  !git clone https://github.com/city96/ComfyUI-GGUF
  !pip install -r ComfyUI-GGUF/requirements.txt

%cd $WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !echo -= Install custom nodes dependencies =-
  !pip install GitPython
  !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies





### Download Resources

In [ ]:
# Download the FLUX model from Hugging Face and save it in the ./models/unet directory
!wget -c "https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q4_K_S.gguf" -P "./models/unet"

# Download the CLIP T5 model from Hugging Face and save it in ./models/clip
!wget -c "https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-Q4_K_S.gguf" -P "./models/clip"

# Download the safetensors file for the CLIP model
!wget -c "https://huggingface.co/f5aiteam/CLIP/resolve/main/clip_l.safetensors" -P "./models/clip"

# Download the VAE model from Hugging Face and save it in ./models/vae
!wget -c "https://huggingface.co/f5aiteam/VAE/resolve/main/ae.safetensors" -P "./models/vae"

# Download the LoRa model from Civitai.com Replace `xxxxxxxx` with your actual token
!wget -c "https://civitai.com/api/download/models/1198851?type=Model&format=SafeTensor&token=xxxxxxxx" -O "./models/loras/FluxThouS40k.safetensors"


### Run ComfyUI

In [1]:
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

--2025-01-20 21:31:19--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.1.0/cloudflared-linux-amd64.deb [following]
--2025-01-20 21:31:19--  https://github.com/cloudflare/cloudflared/releases/download/2025.1.0/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/2d8414e3-a676-4d22-b0ad-71fffb4149a4?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250120%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250120T213119Z&X-Amz-Expires=300&X-Amz-Signature=cbf21cb78cf6c969e24f34289c55412041cf1cf12dc43bded51c4509aea6

# INIT

In [2]:
# @title Environment Setup

from pathlib import Path
import os

# --- Konstansok ---
WORKSPACE_FOLDER = "ComfyUI"
CUSTOM_NODES_FOLDER = "custom_nodes"

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_GGUF_NODE = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['INSTALL_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_GGUF_NODE'] = INSTALL_GGUF_NODE
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES



# Get current directory
current_dir = os.getcwd()
WORKSPACE = f"{current_dir}/ComfyUI"

# Mount Google Drive if needed
def mount_google_drive(OPTIONS):
    if OPTIONS['USE_GOOGLE_DRIVE']:
        print("Mounting Google Drive...")
        os.chdir('/')

        from google.colab import drive
        drive.mount('/content/drive')

        WORKSPACE = "/content/drive/MyDrive/ComfyUI"
        os.chdir('/content/drive/MyDrive')
    else:
      current_dir = os.getcwd()
      workspace = f"{current_dir[0]}/{WORKSPACE_FOLDER}"
    return workspace

# Clone ComfyUI repository if not already cloned
def load_comfyUI(OPTIONS, WORKSPACE):
    if not os.path.exists(WORKSPACE):
        print("Initial setup ComfyUI")
        os.system(f"git clone https://github.com/comfyanonymous/ComfyUI {WORKSPACE}")

    os.chdir(WORKSPACE)

# Update ComfyUI if needed
def update_comfyUI(OPTIONS):
    if OPTIONS['UPDATE_COMFY_UI']:
        print("Updating ComfyUI")
        os.system("chmod -R 755 .ci/nightly/update_windows")
        os.system("chmod -R 755 .ci/update_windows")
        os.system("git pull")

# Install dependencies
def install_dependencies():
    print("Install dependencies")
    os.system("pip3 install accelerate einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3")
    os.system("pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121")
    os.system("pip3 install torchsde kornia>=0.7.1 spandrel soundfile sentencepiece")

# Use ComfyUI Manager if needed
def setup_comfyui_manager(OPTIONS, WORKSPACE):
    if OPTIONS['INSTALL_COMFYUI_MANAGER']:
        os.chdir(f"{WORKSPACE}/custom_nodes")
        os.system("chmod -R 755 ComfyUI-Manager/check.sh ComfyUI-Manager/scan.sh ComfyUI-Manager/node_db/dev/scan.sh ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh ComfyUI-Manager/scripts/install-comfyui-venv-win.bat")

        if not os.path.exists("ComfyUI-Manager"):
            print("Initial setup ComfyUI-Manager")
            os.system("git clone https://github.com/ltdrdata/ComfyUI-Manager")

        os.chdir("ComfyUI-Manager")
        os.system("git pull")
        
# --- GGUF Node Telepítése ---
def install_gguf_node(options, workspace):
  if options['INSTALL_GGUF_NODE']:
      print("-= Install GGUF Node =-")
      custom_nodes_path = os.path.join(workspace, CUSTOM_NODES_FOLDER)
      os.chdir(custom_nodes_path)
      os.system("git clone https://github.com/city96/ComfyUI-GGUF")
      os.system("pip install -r ComfyUI-GGUF/requirements.txt")
      os.chdir(workspace) # Vissza a ComfyUI mappába
      
# --- Custom Node Függőségek Telepítése ---
def install_custom_nodes_dependencies(options, workspace):
  if options['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
    print("-= Install custom nodes dependencies =-")
    os.chdir(workspace)
    os.system("pip install GitPython")
    os.system("python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies")
        
def install_custom_node(options, workspace, node_name, repo_url):
    if not options.get(f'INSTALL_{node_name.upper().replace("-", "_")}', False):
        print(f"Telepítés kihagyva: {node_name}")
        return
    print(f"\n-= *********************************** =-")
    print(f"-= Install {node_name} =-")
    print(f"-= *********************************** =-")
    
    custom_nodes_path = os.path.join(workspace, CUSTOM_NODES_FOLDER)
    
    # custom_nodes mappa létrehozása (ha még nincs)
    if not os.path.exists(custom_nodes_path):
        os.makedirs(custom_nodes_path)
    print(f"custom_nodes mappa elérési útvonala: {custom_nodes_path}")

    destination_path = os.path.join(custom_nodes_path, node_name)

    # Repository klónozása
    if not os.path.exists(destination_path):
        os.system(f"git clone {repo_url} {destination_path}")
        print(f"{node_name} telepítése kész.")
    else:
        print(f"{node_name} már telepítve van.")

    # Függőségek telepítése
    requirements_path = os.path.join(destination_path, "requirements.txt")
    if os.path.exists(requirements_path):
      try:
        os.system(f"pip install -r {requirements_path}")
        print("Dependenciák telepítve.")
      except Exception as e:
         print(f"Hiba a függőségek telepítése során: {e}")
    else:
       print("Nincs requirements.txt fájl, nem szükséges dependenciákat telepíteni.")

    print("Telepítés befejezve.")
    os.chdir(workspace) # Vissza a ComfyUI mappába


   
    
# --- Main ---
if __name__ == '__main__':
  workspace = mount_google_drive(OPTIONS)
  load_comfyUI(OPTIONS, workspace)
  update_comfyUI(OPTIONS)
  setup_comfyui_manager(OPTIONS, workspace)
  install_gguf_node(OPTIONS, workspace)
  install_custom_nodes_dependencies(OPTIONS, workspace)

  
print("-= Install nodes =-")
install_custom_node(OPTIONS, workspace, "ComfyUI-Crystools", "https://github.com/crystian/ComfyUI-Crystools.git")
# install_custom_node(OPTIONS, workspace, "ComfyUI-Easy-Use", "https://github.com/yolain/ComfyUI-Easy-Use.git")

# install_custom_node(OPTIONS, workspace, "ComfyUI-iTools", "https://github.com/fecsaba/ComfyUI-iTools.git")
# install_custom_node(OPTIONS, workspace, "rgthree-comfy", "https://github.com/rgthree/rgthree-comfy.git")
# install_custom_node(OPTIONS, workspace, "comfyui_controlnet_aux", "https://github.com/Fannovel16/comfyui_controlnet_aux.git")
# install_custom_node(OPTIONS, workspace, "ControlAltAI-Nodes", "https://github.com/gseth/ControlAltAI-Nodes.git")

#   print("\n-= **************** =-")
#   print("-= Install Omnigen =-")
#   print("-= **************** =-")
#   print()
os.chdir(workspace) # Vissza a ComfyUI mappába

print("\nTelepítési folyamat befejeződött. Indítsd újra a ComfyUI-t!")

Mounting Google Drive...
Mounted at /content/drive
Initial setup ComfyUI
Updating ComfyUI
Install dependencies
Initial setup ComfyUI-Manager
Install custom nodes dependencies
